# Overview

A **Search Space** is a definition of all the posisble combinations of our models and coresponding hyperparameters through which we will search for the model and hyperparameters which yield the best predictions.

There are many ways to define a search space and these definitions are typically tied to the algorithm used for the search. For example, when performing a grid search on might define the set of possible point (the grid) or one might have a function that returns the next point (rather than all points) to be more efficient.

When doing a random search, search spaces must be defined in a way that ties a probability distribution to a parameter in the search space. Recall that in order for a random search to randomly select a value, it must know the distribution by which to make the selection.

Recall that random variables defined according to a probability distribution are also referred to as stochastic variable. As we will see, the hyperopt framework defines the hyperparameters as stochastic variables using stochastic expressions.

It's interesting that by defining the distribution, one is also defining how the search space will be explored. Thinking about the normal distribution for example, we know that the samples taken from the distribution will center around the mean. We have the possibility to explore points outside the mean, but those opportunities will be less likely as you get further from the mean.

# 1. Parameter Expressions
The hyperopt framework provides **parameter expressions** as a way for a user to define stochastic variables.

```
space = expression(label, parameters)
```

**Note**: The label must be globally unique or you may see a *DuplicateLabel* Exception get raised. The label helps the hyperopt framework define a graph like structure through which the search is conducted. If the labels were not uniquely identified, the framework woudl not be able to understand what parameters it is looking for. This will become more clear as we look at the complex nexted examples

There are two types of parameter expressions which we can leverage:
- **Stochastic Expressions** - Define the hyperparameter using built in distributions
- **Pyll Expressions** - Define the hyperparamter using a user defined distirbution

## 1.1. Parameter Expressions Using Built-In Distributions
Hyperopt has a number of built in distributions which we can use to define our hyperparameters. These include:
- choice
- pchoice
- randint
- uniform
- quniform
- quniformint
- loguniform
- qloguniform
- normal
- qnormal
- lognormal
- qlognormal

All of these parameter expressions are found in the hyperopt.hp module. After looking through the code and making some inferences, I believe that hp stands for hyperparemter. Thus these expressions are hyperparameter expressions.

We will take a closer look at these individually in the next few sections.

### 1.1.1. The Choice Expression
With the choice expression, we can define a choice as a set of posisble outcomes with no specific distribution attached to it.

#### 1.1.1.1. A Univariate Choice
Below we can define a seach space as a binary choice. In this simple case we are choosing between two models (generically named "model a" and "model b").

In [1]:
import hyperopt

space = hyperopt.hp.choice('my_choice', [
    {'name': 'model a'},
    {'name': 'model b'}
])

We can use the hyperopt framework to sable from this search space and examine the arguments that would be provided to our objective function.

In [2]:
print(hyperopt.pyll.stochastic.sample(space))
print(hyperopt.pyll.stochastic.sample(space))
print(hyperopt.pyll.stochastic.sample(space))

{'name': 'model a'}
{'name': 'model b'}
{'name': 'model b'}


We can define a loss function and search through our parameter space for the optimal value using the hyperopt framework:

In [3]:
import hyperopt
import numpy

# Define the objective function
def objective(args):
    print(args)
    if args["name"] == "model a":
        return 0
    else:
        return 1

# Define an object to keep track of the "trials" in the search path
trials = hyperopt.Trials()
    
# Optimize the search space and retrieve the index which points to the best points in the search space
optimal_args_index = hyperopt.fmin(objective, space, algo=hyperopt.tpe.suggest, max_evals=10, trials=trials, rstate= numpy.random.RandomState(42))
    
# Retrieve the resulting hyperparameter set from the search space using the index
optimal_hyperparams = hyperopt.space_eval(space, optimal_args_index)

# Print the results
print("=========================")
print("Optimal args index:")
print(optimal_args_index)
print("Best hyperparameters:")
print(optimal_hyperparams)

{'name': 'model b'}                                                            
{'name': 'model a'}                                                            
{'name': 'model a'}                                                            
{'name': 'model a'}                                                            
{'name': 'model b'}                                                            
{'name': 'model b'}                                                            
{'name': 'model a'}                                                            
{'name': 'model a'}                                                            
{'name': 'model a'}                                                            
{'name': 'model a'}                                                            
100%|██████████████████████| 10/10 [00:00<00:00, 125.00trial/s, best loss: 0.0]
Optimal args index:
{'my_choice': 0}
Best hyperparameters:
{'name': 'model a'}


We can see that "model a" was selected as it yields the minimal results from the objective function.

**Note:** We see that the search algorithm has a lot of repetitions... This is because it the only boundary on the search is the max_evals parameter. We will look at optimizing the search algorithm later on. For example when we utilize the loss_threshold to allow for early termination.

Having a look at the trials opject we inspect it's type and the useful properties.

In [4]:
trials

In [5]:
dir(trials)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_dynamic_trials',
 '_exp_key',
 '_ids',
 '_insert_trial_docs',
 '_trials',
 'aname',
 'argmin',
 'assert_valid_trial',
 'asynchronous',
 'attachments',
 'average_best_error',
 'best_trial',
 'count_by_state_synced',
 'count_by_state_unsynced',
 'delete_all',
 'fmin',
 'idxs',
 'idxs_vals',
 'insert_trial_doc',
 'insert_trial_docs',
 'losses',
 'miscs',
 'new_trial_docs',
 'new_trial_ids',
 'refresh',
 'results',
 'source_trial_docs',
 'specs',
 'statuses',
 'tids',
 'trial_attachments',
 'trials',
 'vals',
 'view']

In [6]:
trials.trials[:2]

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': 1.0, 'status': 'ok'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'my_choice': [0]},
   'vals': {'my_choice': [1]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2021, 11, 24, 23, 27, 44, 377000),
  'refresh_time': datetime.datetime(2021, 11, 24, 23, 27, 44, 377000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': 0.0, 'status': 'ok'},
  'misc': {'tid': 1,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'my_choice': [1]},
   'vals': {'my_choice': [0]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2021, 11, 24, 23, 27, 44, 377000),
  'refresh_time': datetime.datetime(2021, 11, 24, 23, 27, 44, 387000)}]

#### 1.1.1.2. A Nested Univariate Choice
In the example below, we define the search space as a choice between two models. Each model accepts a single hyperparameter which ranges in values.

In [7]:
import hyperopt
import numpy

# Define the search space
space = hyperopt.hp.choice('my_choice', [
    {
        'name': 'model a',
        'x': hyperopt.hp.choice('model_a_x', [0,2,4,6,8])
    },
    {
        'name': 'model b',
        'x': hyperopt.hp.choice('model_b_x', [1,3,5,7,9])
    }
])

# Define the objective function
def objective(args):
    x = args['x']
    return x

# Define an object to keep track of the "trials" in the search path
trials = hyperopt.Trials()
    
# Optimize the search space and retrieve the index which points to the best points in the search space
optimal_args_index = hyperopt.fmin(objective, space, algo=hyperopt.tpe.suggest, max_evals=10, trials=trials, rstate= numpy.random.RandomState(42))
    
# Retrieve the resulting hyperparameter set from the search space using the index
optimal_hyperparams = hyperopt.space_eval(space, optimal_args_index)

# Print the results
print("=========================")
print("Optimal args index:")
print(optimal_args_index)
print("Best hyperparameters:")
print(optimal_hyperparams)

100%|██████████████████████| 10/10 [00:00<00:00, 333.33trial/s, best loss: 0.0]
Optimal args index:
{'model_a_x': 0, 'my_choice': 0}
Best hyperparameters:
{'name': 'model a', 'x': 0}


#### 1.1.1.3. A multivariate nested choice
In the next example we will make things a bit more interesting. We will use models which accept a different set of hyper parameters (one of them being multivariate).

In [8]:
# Define the search space
space = hyperopt.hp.choice('my_choice', [
    {
        'name': 'model a',
        'x': hyperopt.hp.choice('model_a_x', [1,2,4,6,8])
    },
    {
        'name': 'model b',
        'x': hyperopt.hp.choice('model_b_x', [0,1,2,3,4]),
        'y': hyperopt.hp.choice('model_b_y', [0,3,5,7,9])        
    }
])

# Define the objective function
def objective(args):
    x = args['x']
    y = args['y'] if 'y' in args.keys() else 0
    return x + y

# Define an object to keep track of the "trials" in the search path
trials = hyperopt.Trials()
    
# Optimize the search space and retrieve the index which points to the best points in the search space
optimal_args_index = hyperopt.fmin(objective, space, algo=hyperopt.tpe.suggest, max_evals=1000, trials=trials, rstate= numpy.random.RandomState(42), loss_threshold=0.1)
    
# Retrieve the resulting hyperparameter set from the search space using the index
optimal_hyperparams = hyperopt.space_eval(space, optimal_args_index)

# Print the results
print("=========================")
print("Optimal args index:")
print(optimal_args_index)
print("Best hyperparameters:")
print(optimal_hyperparams)

 62%|███████████▋       | 618/1000 [00:05<00:03, 104.57trial/s, best loss: 0.0]
Optimal args index:
{'model_b_x': 0, 'model_b_y': 0, 'my_choice': 1}
Best hyperparameters:
{'name': 'model b', 'x': 0, 'y': 0}


**Note:** We had to significantly increase the mas_evals and add the loss threshold param to allow us to terminate early if the loss is less than the threshold supplied.

#### 1.1.1.4. A Conditional Parameters
In the documentation we see an example of a conditional parameter. The idea is that a varibale is used if come condition is met. In the example below 'c1' and 'c2' are conditional parameters. Each of 'c1' and 'c2' only figures in the returned sample for a particular value of 'a'. If 'a' is 0, then 'c1' is used but not 'c2'. If 'a' is 1, then 'c2' is used but not 'c1'. Whenever it makes sense to do so, you should encode parameters as conditional ones this way, rather than simply ignoring parameters in the objective function. If you expose the fact that 'c1' sometimes has no effect on the objective function (because it has no effect on the argument to the objective function) then search can be more efficient about credit assignment.

In [9]:
# Define the search space
space = hyperopt.hp.choice('x',
    [
        ('case 1', 1 + hyperopt.hp.lognormal('c1', 0, 1)),
        ('case 2', hyperopt.hp.uniform('c2', -10, 10))
    ])

# Define the objective function
def objective(args):
    case, value = args
    return value

# Define an object to keep track of the "trials" in the search path
trials = hyperopt.Trials()
    
# Optimize the search space and retrieve the index which points to the best points in the search space
optimal_args_index = hyperopt.fmin(objective, space, algo=hyperopt.tpe.suggest, max_evals=1000, trials=trials, rstate= numpy.random.RandomState(42), loss_threshold=0.1)
    
# Retrieve the resulting hyperparameter set from the search space using the index
optimal_hyperparams = hyperopt.space_eval(space, optimal_args_index)

# Print the results
print("=========================")
print("Optimal args index:")
print(optimal_args_index)
print("Best hyperparameters:")
print(optimal_hyperparams)

  0%|       | 1/1000 [00:00<02:19,  7.14trial/s, best loss: -7.032761300708053]
Optimal args index:
{'c2': -7.032761300708053, 'x': 1}
Best hyperparameters:
('case 2', -7.032761300708053)


#### 3.2.1.2. The Randint Expression
The Randint Expression allows us to define a hyperparameter as a random integer.

In [10]:
import hyperopt

# define the upper limit of the domain for the random integer
space = hyperopt.hp.randint('my_choice', 10)

In [11]:
print(hyperopt.pyll.stochastic.sample(space))
print(hyperopt.pyll.stochastic.sample(space))
print(hyperopt.pyll.stochastic.sample(space))

3
2
9


Incorporating this into our example decision we have: 

In [12]:
import hyperopt
import numpy

# Define the search space
space = hyperopt.hp.choice('my_choice', [
    {
        'name': 'model a',
        'x': hyperopt.hp.randint('model_a_x', 10)
    },
    {
        'name': 'model b',
        'x': hyperopt.hp.randint('model_b_x', 15)
    }
])

# Define the objective function
def objective(args):
    x = args['x']
    return x

# Define an object to keep track of the "trials" in the search path
trials = hyperopt.Trials()
    
# Optimize the search space and retrieve the index which points to the best points in the search space
optimal_args_index = hyperopt.fmin(objective, space, algo=hyperopt.tpe.suggest, max_evals=10, trials=trials, rstate= numpy.random.RandomState(42))
    
# Retrieve the resulting hyperparameter set from the search space using the index
optimal_hyperparams = hyperopt.space_eval(space, optimal_args_index)

# Print the results
print("=========================")
print("Optimal args index:")
print(optimal_args_index)
print("Best hyperparameters:")
print(optimal_hyperparams)

100%|██████████████████████| 10/10 [00:00<00:00, 500.00trial/s, best loss: 0.0]
Optimal args index:
{'model_a_x': 0, 'my_choice': 0}
Best hyperparameters:
{'name': 'model a', 'x': 0}


## 1.2. User Defined Expresisons with Pyll
In some cases the built-in distributions may not provide the descriptive capabilities we want to model our hyperparameter. for example if we wanted to express our random variable as:

$$ Z = X + Y; \ \ where \ X \sim \mathcal{N}, Y \sim \mathcal{U}  $$


To accomodate this need, Hyperopt gives us the ability to write complex expressions in two ways:
- Embedding an expression in a search space
- Using a Pyll function

We will see examples of these below

### 1.2.1. Embedding an expression in a search space

In [13]:
import hyperopt
import numpy

# Define the search space
space = hyperopt.hp.choice('my_choice', [
    {
        'name': 'model a',
        'x': hyperopt.hp.choice('model_a_x', [1,2,3,4,5])
    },
    {
        'name': 'model b',
        'x': hyperopt.hp.uniform('model_b_x', -10, 10) + hyperopt.hp.uniform('model_b_y', -5, 5)
    }
])

# Define the objective function
def objective(args):
    x = args['x']
    return x

# Define an object to keep track of the "trials" in the search path
trials = hyperopt.Trials()
    
# Optimize the search space and retrieve the index which points to the best points in the search space
optimal_args_index = hyperopt.fmin(objective, space, algo=hyperopt.tpe.suggest, max_evals=10000, trials=trials, rstate= numpy.random.RandomState(42), loss_threshold=0.1)
    
# Retrieve the resulting hyperparameter set from the search space using the index
optimal_hyperparams = hyperopt.space_eval(space, optimal_args_index)

# Print the results
print("=========================")
print("Optimal args index:")
print(optimal_args_index)
print("Best hyperparameters:")
print(optimal_hyperparams)

  0%|             | 1/10000 [00:00<?, ?trial/s, best loss: -0.7696510873187563]
Optimal args index:
{'model_b_x': 2.74672956303527, 'model_b_y': -3.5163806503540265, 'my_choice': 1}
Best hyperparameters:
{'name': 'model b', 'x': -0.7696510873187563}


### 1.2.2. Using a Pyll function
The hyperopt library provides the pyll module which allows parameterized function to be placed into search spaces.

In [14]:
import hyperopt
import numpy

# Define a deterministic function to use with pyll
def foobar(x, y):
    return x + y

# Define the search space
space = hyperopt.hp.choice('my_choice', [
    {
        'name': 'model a',
        'x': hyperopt.hp.choice('model_a_x', [1,2,3,4,5])
    },
    {
        'name': 'model b',
        'x': hyperopt.pyll.scope.call(foobar, (
            hyperopt.hp.uniform('model_b_x', -10, 10), 
            hyperopt.hp.uniform('model_b_y', -5, 5)
        ))
    }
])

# Define the objective function
def objective(args):
    x = args['x']
    return x

# Define an object to keep track of the "trials" in the search path
trials = hyperopt.Trials()
    
# Optimize the search space and retrieve the index which points to the best points in the search space
optimal_args_index = hyperopt.fmin(objective, space, algo=hyperopt.tpe.suggest, max_evals=10000, trials=trials, rstate= numpy.random.RandomState(42), loss_threshold=0.1)
    
# Retrieve the resulting hyperparameter set from the search space using the index
optimal_hyperparams = hyperopt.space_eval(space, optimal_args_index)

# Print the results
print("=========================")
print("Optimal args index:")
print(optimal_args_index)
print("Best hyperparameters:")
print(optimal_hyperparams)

  0%|    | 1/10000 [00:00<01:39, 100.00trial/s, best loss: -0.7696510873187563]
Optimal args index:
{'model_b_x': 2.74672956303527, 'model_b_y': -3.5163806503540265, 'my_choice': 1}
Best hyperparameters:
{'name': 'model b', 'x': -0.7696510873187563}
